In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as sorters
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
import settings
import OpenEphys
import numpy as np
import pandas as pd
import pickle

16:34:43 [I] klustakwik KlustaKwik2 version 0.2.6


In [2]:
recording_location = 'testData/M1_D31_2018-11-01_12-28-25/'
def load_OpenEphysRecording(folder):
    signal = []
    for i in range(settings.num_tetrodes*4):
        fname = folder+settings.data_file_prefix+str(i+1)+settings.data_file_suffix+'.continuous'
        x = OpenEphys.loadContinuousFast(fname)['data']
        if i==0:
            #preallocate array on first run
            signal = np.zeros((settings.num_tetrodes*4,x.shape[0]))
        signal[i,:] = x
    return signal

signal = load_OpenEphysRecording(recording_location)

geom = pd.read_csv('sorting_files/geom_all_tetrodes_original.csv',header=None).values


Fs = settings.sampling_rate
recording =se.NumpyRecordingExtractor(signal,settings.sampling_rate,geom)

Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...


In [12]:
recording_prb = recording.load_probe_file('sorting_files/tetrode_16.prb')
recording_f = st.preprocessing.bandpass_filter(recording_prb, freq_min=300, freq_max=6000,type='butter',chunk_size=500)

In [13]:
with open('sorting_ms4.pkl','rb') as f:
    sorting_ms4 = pickle.load(f)

In [15]:
%lprun -f st.preprocessing.BandpassFilterRecording._do_filter st.curation.threshold_snr(sorting=sorting_ms4, recording = recording_f, threshold = 5, threshold_sign='less', max_snr_waveforms=50)

Timer unit: 1e-06 s

Total time: 3.45722 s
File: /Users/teristam/anaconda3/envs/mountainlab/lib/python3.6/site-packages/spiketoolkit/preprocessing/bandpass_filter.py
Function: _do_filter at line 80

Line #      Hits         Time  Per Hit   % Time  Line Contents
    80                                               def _do_filter(self, chunk):
    81      1075       5664.0      5.3      0.2          samplerate = self._recording.get_sampling_frequency()
    82      1075       2547.0      2.4      0.1          M = chunk.shape[0]
    83      1075        578.0      0.5      0.0          chunk2 = chunk
    84                                                   # Do the actual filtering with a DFT with real input
    85      1075       1254.0      1.2      0.0          if self._type == 'fft':
    86                                                       chunk_fft = np.fft.rfft(chunk2)
    87                                                       kernel = self._create_filter_kernel(
    88         

In [63]:
snr=st.curation.threshold_snr(sorting=sorting_ms4, recording = recording_f, threshold = 5, threshold_sign='less', max_snr_waveforms=2)

hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit


In [65]:
snr.get_epoch()

TypeError: get_epoch() missing 1 required positional argument: 'epoch_name'

In [9]:
from scipy.signal import butter
from scipy.signal import filtfilt
import scipy.signal as signal

y = recording._timeseries
fn = recording.get_sampling_frequency() / 2.
band = np.array([300, 6000]) / fn
b, a = butter(3, band, btype='bandpass')
if not (np.all(np.abs(np.roots(a)) < 1) and np.all(np.abs(np.roots(a)) < 1)):
    raise ValueError('Filter is not stable')


In [14]:
%timeit signal.filtfilt(b,a,y[:,:30000*120],axis=1)

1.56 s ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
y.shape

(16, 95519744)